# Trialing the logistic regression model,
The code will run the logistic regression model on the FTSE 350 stock data and see what the profit would have been from investing with the below criteria:
- Starting with \u00c2\u00a310,000
- Never investing more than 10% of the value into a single stock
- Tracking balance so once the account is empty no more can be invested until shares are liqidated
- Shares are bought at the open of the day following the first buy signal, if shares are not already held
- Shares are sold at the open of the day following the first close signal after a hold period, if shares are held

Trading variables:
- Trades cost \u00c2\u00a32.50 to execute
- Spread is 1%

In [1]:
#Import and combine prices files
path = r'C:\\Users\\Robert\\Documents\\Python Scripts\\Stock trading - ML modelling\\Historical prices'
df_ft = pd.read_hdf(path +'\\all_hist_prices_w_ft_eng2.h5')
print('SHAPE: {}'.format(df_ft.shape))
print(df_ft.columns)
df_ft.head()

IndentationError: unexpected indent (<ipython-input-1-427a504bc45f>, line 2)

In [ ]:
#Import the features
file_object = open(path+r'\\log_reg_model_feature_list.txt','r')
ft_li = file_object.read().split(',')
file_object.close()
ft_li

In [2]:
#Import to lr model
lr_mod = jl.load(path+r'\\log_reg_model.joblib')
lr_mod

IndentationError: unexpected indent (<ipython-input-2-a8d39a58587a>, line 2)

In [3]:
lr_mod_df = df_ft.replace([np.inf,-np.inf],np.nan).dropna(axis=0)
lr_mod_df.head()

IndentationError: unexpected indent (<ipython-input-3-2b12aeef3884>, line 2)

In [4]:
#Run the lr model on the data
lr_mod_df = df_ft.replace([np.inf,-np.inf],np.nan).dropna(axis=0).reset_index(drop=True) #Reset index allows joining of signal after calc_probs_df

#Function for creating a dataframe with model probabilities and the most likely outcome
def calc_probs_df(models,unique_classes,df_test,features):
    modelled_probs = pd.DataFrame(columns=unique_classes)
    for cl in unique_classes:
        x_test = df_test[features]
        modelled_probs[cl] = models[cl].predict_proba(x_test)[:,1] 
        #Column index 1 as predict_proba outputs 2 columns,
            #the first is the probability that it is a negative result
            #the second is the probability that it is a positive result
            #that's why we want column index 1, we are looking at the positive result
    #Determine the most probable result
    modelled_probs['signal'] = modelled_probs.idxmax(axis=1)
    modelled_probs['signal_prob'] = modelled_probs[unique_classes].max(axis=1)
#     print(modelled_probs)
    for cl in unique_classes:
        print(len(modelled_probs.loc[modelled_probs['signal'] == cl,:]))
    return (modelled_probs['signal'],modelled_probs['signal_prob'])
lr_mod_df['signal'],lr_mod_df['signal_prob'] = calc_probs_df(lr_mod,['buy','sell','hold'],lr_mod_df,ft_li)
lr_mod_df.head()

IndentationError: unexpected indent (<ipython-input-4-32882f9e448d>, line 2)

In [5]:
#Import and combine prices files
df_prices = pd.read_hdf(path +'\\all_hist_prices_w.h5')

IndentationError: unexpected indent (<ipython-input-5-25589bcb4019>, line 2)

In [6]:
#Sort by ticker and date then add the open_shift_neg1 field
#These allow the buying and selling to be done at a realistic price
df_prices.sort_values(['Ticker','Date'],ascending=[True,True],inplace=True)
df_prices['open_shift_neg1'] = df_prices['Open'].shift(-1)
df_prices['Date'] = df_prices['Date'].astype('datetime64')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
df_prices.head()

IndentationError: unexpected indent (<ipython-input-6-a94dda670419>, line 2)

In [7]:
#Join on the buy and sell signals
df_prices = pd.merge(df_prices[['Ticker','Date','Open','Close','open_shift_neg1']],lr_mod_df[['Ticker','Date','signal','signal_prob']],left_on=['Ticker','Date'],right_on=['Ticker','Date'],how='inner')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
df_prices.head()

IndentationError: unexpected indent (<ipython-input-7-1f1b27acae3d>, line 2)

In [8]:
#Create a class for maintaining the value of the fund
class new_fund:
    def __init__(self,init_val):
        self.st_val = init_val
        self.available = init_val
        self.invested_value = 0
        self.codb = 0
        self.ledger = []
        self.cur_holdings = {}
        print('NEW FUND')
        print('st_val:{}'.format(self.st_val))
        print('available:{}'.format(self.available))
    @property
    def fund_value(self):
        return self.invested_value + self.available
    #Create a function to buy shares
    def buy(self,ticker:str,price:float,spread:float,value:float,trade_cost:float,val_inc_tc:bool = True):
#         print('')
#         print('BUY SHARE:{}'.format(ticker))
#         print('PRICE:{}'.format(price))
#         print('SPREAD:{}'.format(spread))
#         print('VALUE:{}'.format(value))
#         print('TRADE COST:{}'.format(trade_cost))
#         print('VALUE INC TRADE COST:{}'.format(val_inc_tc))
#         #Check if already bought
        if ticker in self.cur_holdings:
#             print('{} ALREADY IN HOLDINGS, EXITING BUY METHOD'.format(ticker))
            return
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {}'.format(price))
        #Calculate the trade value
        if val_inc_tc == True:
            trade_funds = value - trade_cost
        else:
            trade_funds = value
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the number of whole shares which can be purchased
        share_vol = int(trade_funds/a_price)
        trade_value = round(share_vol * a_price,2)
        #Calc the total_spend
        spread_cost = round(share_vol * price * spread,2)
        total_spend = (share_vol*a_price) + trade_cost
        #Check the fund has the money to cover this trade
        if total_spend > self.available:
            raise ValueError('you do not have the funds to make this trade -> this transaction will be cancelled')
        #Create a record for the ledger
        ledge_rec = {
            'trade_type':'buy\
            ,'ticker':ticker
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'total_spend':total_spend
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value + (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available - total_spend
        }
        self.ledger.append(ledge_rec)
        #Update the object
        self.available += -round(total_spend,2)
        self.codb += round(trade_cost + spread_cost,2)
        self.invested_value += round(share_vol*price,2)
        #Add to cur_holdings
        holding_rec = {
           'share_vol':share_vol
            ,'cur_price':price
            ,'value':round(share_vol*price,2)
        }
        #Check if key already in dict
        if ticker in self.cur_holdings:
#             print('{} IS IN cur_holdings'.format(ticker))
            self.cur_holdings[ticker]['share_vol'] += share_vol
            self.cur_holdings[ticker]['cur_price'] = round(price,2)
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
        else:
#             print('{} IS NOT IN cur_holdings'.format(ticker))
            self.cur_holdings[ticker] = holding_rec
#         print('a_price:{}'.format(a_price))
#         print('b_price:{}'.format(b_price))
#         print('trade_funds:{}'.format(trade_funds))
#         print('share_vol:{}'.format(share_vol))
#         print('trade_value:{}'.format(trade_value))
#         print('self.available:{}'.format(self.available))
#         print('self.codb:{}'.format(self.codb))
#         print('self.invested_value:{}'.format(self.invested_value))
#         print('self.fund_value:{}'.format(self.fund_value))
        
    #Create a function to sell shares
    def sell(self,ticker:str,trade_date:datetime,price:float,spread:float,trade_cost:float):
#         print('')
#         print('SELL SHARE:{}'.format(ticker))
#         print('PRICE:{}'.format(price))
#         print('SPREAD:{}'.format(spread))
        #Check if already sold
        if ticker not in self.cur_holdings:
#             print('{} NOT IN HOLDINGS, EXITING SELL METHOD'.format(ticker))
            return
        #Round down the share_vol
        share_vol = int(self.cur_holdings[ticker]['share_vol'])
#         print('SHARE VOL:{}'.format(share_vol))
#         print('TRADE COST:{}'.format(trade_cost))
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {}'.format(price))
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the trade value
        trade_value = round(share_vol*b_price,2)
        value = round(trade_value - trade_cost,2)
        spread_cost = round(share_vol * price * spread)
        #Calc the total_spend
        total_spend = round(trade_cost+spread_cost-(share_vol*price),2)
        #Check the fund has the money to cover this trade
        if total_spend > self.available:
            raise ValueError('you do not have the funds to make this trade -> this transaction will be cancelled')
        #Create a record for the ledger
        ledge_rec = {
           'trade_type':'sell\
            ,'ticker':ticker
            ,'trade_date':ticker
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'total_spend':total_spend
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value - (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available - total_spend
        }
        self.ledger.append(ledge_rec)
        #Update the object
        self.available += -round(total_spend,2)
        self.codb += round(trade_cost + spread_cost,2)
        self.invested_value += -round((share_vol*price),2)
        #Remove from cur_holdings
        #Check if key already in dict
        if ticker in self.cur_holdings:
#             print('{} IS IN cur_holdings'.format(ticker))
            if self.cur_holdings[ticker]['share_vol'] > share_vol:
                self.cur_holdings[ticker]['share_vol'] += -share_vol
                self.cur_holdings[ticker]['cur_price'] = price
                self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
            elif self.cur_holdings[ticker]['share_vol'] == share_vol:
                del self.cur_holdings[ticker] #Delete from the dictionary
            else:
                raise ValueError('you do not have enough share to make this trade. You want to sell {} of {} however you only have {}'.format(share_vol,ticker,self.cur_holdings[ticker]['share_vol']))
        else:
#             print('{} IS NOT IN cur_holdings'.format(ticker))
            return
#         print('a_price:{}'.format(a_price))
#         print('share_vol:{}'.format(share_vol))
#         print('trade_value:{}'.format(trade_value))
#         print('self.available:{}'.format(self.available))
#         print('self.codb:{}'.format(self.codb))
#         print('self.invested_value:{}'.format(self.invested_value))
#         print('self.fund_value:{}'.format(self.fund_value))
#         print('self.cur_holdings:{}'.format(self.cur_holdings))
        
    #Create a function to update value after a price change
    def price_change(self,ticker:str,price:float):
#         print('')
#         print('PRICE CHANGE:{}'.format(ticker))
#         print('PRICE:{}'.format(price))
        #Check if key already in dict
        if ticker in self.cur_holdings:
            #Update the object
            self.invested_value += round((self.cur_holdings[ticker]['share_vol']*price) - self.cur_holdings[ticker]['value'],2)
#             print('{} IS IN cur_holdings'.format(ticker))
            self.cur_holdings[ticker]['cur_price'] = price
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
#             print('price:{}'.format(price))
#             print('value:{}'.format(self.cur_holdings[ticker]['value']))
        else:
#             print('{} NOT IN HOLDINGS, EXITING SELL METHOD'.format(ticker))
            return
#         print('self.invested_value:{}'.format(self.invested_value))
#         print('self.fund_value:{}'.format(self.fund_value))

IndentationError: unexpected indent (<ipython-input-8-345fe49852c1>, line 2)

In [9]:
#Order the data by date (asc) and buy probability (desc)
signal_df = df_prices[['Ticker','Date','Open','Close','open_shift_neg1','signal','signal_prob']].copy()
signal_df.sort_values(['Date','signal_prob'],ascending=[True,False],inplace=True)
signal_df.reset_index(drop=True,inplace=True)
signal_df.head()

IndentationError: unexpected indent (<ipython-input-9-54576cb6e343>, line 2)

In [10]:
#Establish trading variables
#All figures in pence
fund_value_st = 1000000 #\u00c2\u00a310,000
trade_cost = 250
investment_limit_min_val = 100000 #\u00c2\u00a31,000
investment_limit_max_per = 0.1 #10%
spread = 0.01 #1%

IndentationError: unexpected indent (<ipython-input-10-140bbe0f99c8>, line 2)

In [11]:
#Run through rows and buy and sell according to signals and holdings
fund = new_fund(fund_value_st)
for index,row in signal_df.iterrows():
#     print('')
#     print('NEW ROW: {} {:%d/%m/%Y} {} {:4.4f}'.format(row['Ticker'],row['Date'],row['signal'],row['signal_prob']))
    #Follow signal
    if row['signal'] =='buy':
#         print('FIRE BUY')
        #Check for funds
        if fund.available < investment_limit_min_val:
#             print('NOT ENOUGH FUNDS TO BUY, EXITING -> fund.available:{}, investment_limit_min_val:{}'.format(fund.available,investment_limit_min_val))
            continue
        #Buy shares
        val_to_invest = fund.available*investment_limit_max_per if fund.available*investment_limit_max_per > investment_limit_min_val else investment_limit_min_val
        fund.buy(row['Ticker'],row['open_shift_neg1'],spread,val_to_invest,trade_cost)
    elif row['signal'] =='sell':
#         print('FIRE SELL')
        #Sell shares
        fund.sell(row['Ticker'],row['open_shift_neg1'],spread,trade_cost)
    elif row['signal'] == 'hold':
#         print('FIRE HOLD')
        fund.price_change(row['Ticker'],row['Close'])
        
#     buy(self,ticker:str,price:float,spread:float,value:float,trade_cost:float,val_inc_tc:bool = True)
#     sell(self,ticker:str,price:float,spread:float,trade_cost:float)
#     price_change(self,ticker:str,price:float)
#         self.st_val = init_val
#         self.available = init_val
#         self.invested_value = 0
#         self.codb = 0
#         self.ledger = []
#         self.cur_holdings = {}'

IndentationError: unexpected indent (<ipython-input-11-ce5af6502c90>, line 2)

In [12]:
#Show summary
print('fund.st_val:\u00c2\u00a3{:,.2f}'.format(fund.st_val/100))
print('fund.available:\u00c2\u00a3{:,.2f}'.format(fund.available/100))
print('fund.codb:\u00c2\u00a3{:,.2f}'.format(fund.codb/100))
print('fund.invested_value:\u00c2\u00a3{:,.2f}'.format(fund.invested_value/100))
ledger_df = pd.DataFrame(fund.ledger)
print('TRADE COUNT:{:,}'.format(len(ledger_df)))
ledger_df.head(50)

IndentationError: unexpected indent (<ipython-input-12-25edfe51e8c6>, line 2)

In [ ]:
#From the ledger create a dataframe of completed trades
completed_trades = {}
#Format
#     ABC:{
#         open_position:True/False #Bool showing if there is currently an open position
#         ,trades:[ #List showing all trades
#             { #Each trade has an object
#                 share_vol:12345 #Volume of shares purchased
#                 ,buy_spend:12345.67 #Total value spent in buying shares including costs
#                 ,sell_spend:12345.67 #Total value spent in selling shares including costs
#                 ,profit_loss:12345.67 #Profit/loss of ths trade
#             }
#         ]
#     }
for index,row in ledger_df.iterrows():
    #Check if there is an open trade for this ticker
    if row['ticker'] not in completed_trades:
        completed_trades[row['ticker']] = {
            'open_position':False
            ,'trades':[]
        }
    #Deal with buying
    if row['trade_type'] == 'buy':
        #Create a trade object and add to the trades list in completed_trades
        completed_trades[row['ticker']]['trades'].append({
            'share_vol':row['share_vol']
            ,'buy_spend':row['total_spend']
            ,'sell_spend':None
            ,'profit_loss':None
        })
#         if completed_trades[row['ticker']].open_position == True:
#             #Get the last trade
#             completed_trades[row['ticker']].trades:
trades_li = []
for tick in completed_trades:
    for trade in completed_trades[tick]['trades']:
        trades_li.append(trade)
pd.DataFrame(trades_li)